In [1]:
import gurobipy as gb
import pandas as pd
import gurobipy as gb

In [2]:
df = pd.read_csv("~/Desktop/uni BAM/MS/Group Assignment/flights-1.csv",  encoding='latin-1')

issues = {"ZÃ¼rich": "Zurich",
          "DÃ¼sseldorf": "Dusseldorf", "MÃ¡laga": "Malaga"}
for key, value in issues.items():
    df.loc[df["departureCity"] == key, "departureCity"] = value
    df.loc[df["arrivalCity"] == key, "arrivalCity"] = value

In [3]:
df2 = df.set_index('departureCity').join(df.set_index('departureCity'),
                                         lsuffix = "first", rsuffix = "second").reset_index()
df2 = df2[df2.arrivalCityfirst != df2.arrivalCitysecond]
df2["distance"] = df2.Distancefirst + df2.Distancesecond
df2 = df2[["departureCity", "arrivalCityfirst", "arrivalCitysecond", "distance"]]
df2.columns = ["Hub", "arrivalCity", "departureCity", "Distance"]

In [4]:
merged_df = df2.merge(df[["departureCity", "arrivalCity", "Distance", "Demand"]],
               on=["departureCity", "arrivalCity"],
               how="left")
merged_df.columns = ["Hub", "arrivalCity", "departureCity", "IndirectDistance","DirectDistance", "Demand"]
filtered_df = merged_df[merged_df["IndirectDistance"] <= 1.3* merged_df["DirectDistance"]] # set of all the indirect routes that can be taken and their distances

In [5]:
Indirect_Distance = {}
for index, row in filtered_df.iterrows():
    Indirect_Distance[row.departureCity, row.Hub, row.arrivalCity] = row.IndirectDistance
    
Direct_Distance = {}
for index, row in merged_df.iterrows():
    Direct_Distance[row.departureCity, row.arrivalCity] = row.DirectDistance
    
Direct_Demand = {}
for index, row in merged_df.iterrows():
    Direct_Demand[row.departureCity, row.arrivalCity] = row.Demand

Indirect_Demand = {}
for index, row in filtered_df.iterrows():
    Indirect_Demand[row.departureCity, row.Hub, row.arrivalCity] = row.Demand
    
ItineraryFlights = {}
for index, row in merged_df.iterrows():
    itflights = []
    if row.Hub == "":
        itflights.append((row.departureCity, row.arrivalCity))
        ItineraryFlights[row.departureCity, row.arrivalCity] = itflights
    else:
        itflights.append((row.departureCity, row.Hub))
        itflights.append((row.Hub, row.arrivalCity))
        ItineraryFlights[row.departureCity, row.arrivalCity] = itflights
        
HubCities = list(set(merged_df.Hub.unique()))
HubCities = HubCities[1:]

IndirectRevenuePerCustomer = {}
for index, row in filtered_df.iterrows():
        IndirectRevenuePerCustomer[row.departureCity, row.Hub, row.arrivalCity] = row.DirectDistance*0.08

DirectRevenuePerCustomer = {}
for index, row in merged_df.iterrows():
        DirectRevenuePerCustomer[row.departureCity, row.arrivalCity] = row.DirectDistance * 0.1

In [ ]:
charge_indirect = 0.08
charge_direct = 0.1
cost_per_mile = {"small":4.5, "medium":8, "large":20}
capacity = {"small":50, "medium":100, "large":300}
sizes = ["small", "medium", "large"]
M = 1000000000000000

model = gb.Model()
model.setParam('TimeLimit', 15*60)

# Variables
small_direct = model.addVars(ItineraryFlights, name = "small", vtype = gb.GRB.INTEGER, lb = 0)
medium_direct = model.addVars(ItineraryFlights, name = "medium", vtype = gb.GRB.INTEGER, lb = 0)
large_direct = model.addVars(ItineraryFlights, name = "large", vtype = gb.GRB.INTEGER, lb = 0)

small_indirect = model.addVars(Indirect_Distance, name = "small", vtype = gb.GRB.INTEGER, lb = 0)
medium_indirect = model.addVars(Indirect_Distance, name = "medium", vtype = gb.GRB.INTEGER, lb = 0)
large_indirect = model.addVars(Indirect_Distance, name = "large", vtype = gb.GRB.INTEGER, lb = 0)

x = model.addVars(Direct_Distance ,name = "direct_passengers", vtype = gb.GRB.INTEGER, lb = 0, ub = Direct_Demand)
f = model.addVars(Indirect_Distance, name = "indirect_passengers", vtype = gb.GRB.INTEGER, lb = 0, ub = Indirect_Demand)

h = model.addVars(HubCities, name = "hub", vtype = gb.GRB.BINARY, lb = 0, ub = 1)

# Set Constraints

# There can only be 1 hub
for city in HubCities:
    model.addConstr(gb.quicksum(f[route] for route in Indirect_Distance if city in route) <= h[city] * M)
model.addConstr(gb.quicksum(h[k] for k in HubCities) == 2)

# Total number of passengers cannot exceed demand
#model.addConstrs((x[route]) <= Direct_Demand[route] for route in Direct_Distance)
#model.addConstrs((f[route]) <= Indirect_Demand[route] for route in Indirect_Distance)

model.addConstrs((x[route] + f[routes]) <= Direct_Demand[route] for route in Direct_Distance for routes in Indirect_Distance)

# Number of passengers cannot exceed capacity
#model.addConstrs((x[route]) <= capacity[s] * cost_per_mile[s] for s in sizes for route in Direct_Distance)
#model.addConstrs((f[route]) <= capacity[s] * cost_per_mile[s] for s in sizes for route in Indirect_Distance)

#model.addConstrs((x[route] + f[routes]) <= capacity["small"] * small_direct[route] +
 #                capacity["medium"] * medium_direct[route] +
  #               capacity["large"] * large_direct[route] for route in Direct_Distance for routes in Indirect_Distance)

model.addConstrs(
    gb.quicksum(x[route] + f[indirect_route] for indirect_route in Indirect_Distance[route])
    <= capacity["small"] * small_direct[route] +
       capacity["medium"] * medium_direct[route] +
       capacity["large"] * large_direct[route]
    for route in Direct_Distance
)



# Setting the Objective
model.setObjective(
    gb.quicksum(
        DirectRevenuePerCustomer[route] - Direct_Distance[route] * (
            cost_per_mile["small"] * small_direct[route] +
            cost_per_mile["medium"] * medium_direct[route] +
            cost_per_mile["large"] * large_direct[route]
        ) for route in Direct_Distance
    ) +
    gb.quicksum(
        IndirectRevenuePerCustomer[route] - Indirect_Distance[route] * (
            cost_per_mile["small"] * small_indirect[route] +
            cost_per_mile["medium"] * medium_indirect[route] +
            cost_per_mile["large"] * large_indirect[route]
        ) for route in Indirect_Distance
    ),
    gb.GRB.MAXIMIZE
)

model.optimize()

if model.status == gb.GRB.OPTIMAL:

    selected_hubs = [city for city in HubCities if h[city].x > 0.5]
    print("Selected Hubs:", selected_hubs)
else:
    print("Optimization was not successful.")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
Set parameter TimeLimit to value 900


In [ ]:
vals = model.printAttr(["X", "Obj"])
all_vars = model.getVars()

# Since there are a lot of variables, I decided to move them into a dataframe format
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

res = pd.DataFrame({"names": names, "values": values})


    Variable            X          Obj 
--------------------------------------
small[Madrid,London]            0        -3528 
small[Paris,London]            0       -958.5 
small[Frankfurt,London]            0      -1786.5 
small[Amsterdam,London]            0         -999 
small[Istanbul,London]            0        -7002 
small[Palma de Mallorca,London]            0      -3757.5 
small[Rome,London]            0        -4014 
small[Moscow,London]            0        -7011 
small[Lisbon,London]            0      -4432.5 
small[Zurich,London]            0      -2173.5 
small[Milan,London]            0        -2682 
small[Copenhagen,London]            0      -2677.5 
small[Manchester,London]            0       -733.5 
small[Malaga,London]            0        -4689 
small[Vienna,London]            0      -3460.5 
small[Oslo,London]            0        -3231 
small[Dusseldorf,London]            0      -1345.5 
small[Dublin,London]            0        -1296 
small[Athens,London]           

In [8]:
small_res = res[res["names"].str.startswith("small")]["values"].sum()
medium_res = res[res["names"].str.startswith("medium")]["values"].sum()
large_res = res[res["names"].str.startswith("large")]["values"].sum()

NameError: name 'res' is not defined